In [1]:
%matplotlib inline

import geopandas as gpd

import pandas as pd
import os
import requests
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
import matplotlib.font_manager as fm

for font in fm.fontManager.ttflist:
    if font.name in ['AppleGothic', 'Malgun Gothic']:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['font.family']

['Malgun Gothic']

In [3]:
RouteStation_latlon = pd.read_csv('df_route_stations.csv', encoding='euc-kr')

In [134]:
RouteStation_latlon.sample()

,Unnamed: 0,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no,govern,lat,lng,이비노선ID,표준노선ID
1881,9363,90740,233000080,21,33,고주리,233001510,36736.0,화성시,37.128833,126.883833,233000080.0,41008146.0


In [93]:
RouteStation_latlon['bus_line_no'].unique()

array(['24', '1004', '330', '33-1', '2', '4', '5-2', '5-4', '15', '14',
       '9', '19', '3-1', '3', '2-4', '2-3', '2-2', '16', '18', '4-1', '6',
       '11', '6-2', '17', '6-3', '8', '8-3', '1', '8-2', '1-1', '2-1',
       '7-1', '8-1', '7', '11-5', '11-2', '11-4', '10', '1004-1', '340',
       '1000', '20', '21', '330-1', '23', '38', '25', '26', '27', '340-1',
       '1008', '33-2', '712', '73-1', '6001', '116-3', '1001', '8501',
       '6002', '8155', '1002', '201', '6002-1', '202', '710', '150',
       '203', '8156', '2000A', '2000B', '4403', '6003', '28', '200',
       '8156(급행)', '4108', '9802', '8472', '8471', '39', 'H6005', '116-5',
       'H6007', 'H6006', '205', '720-3', '6004', '721', '116-2', '9-1',
       '31-3', '3-2', '50', '76', '77', '78', '79', '80', '81', '333',
       '709', '709-1', 'H4', 'H5', 'H6(A)', 'H6(B)', '19-1', '19-2', '29',
       '19-3', 'H1', 'H3', 'H2', '20(마산포)', '20(어도동내)', '20(어도펜션)',
       '공영1(사곳.궁평)', '공영1(매화.궁평)', '공영1(궁평.사곳)', '공영1(지화2리)',
  

In [7]:
st_nm = RouteStation_latlon['station_nm'].unique()

In [107]:
st_nm = list(st_nm)
st_nm

['한원IC',
 '장지동',
 '롯데캐슬.제일풍경채',
 '자이파밀리에',
 '꽃가마약수터',
 '장외리.오리골',
 '돈말입구',
 '제부도입구',
 '송교리(경유)',
 '개경촌',
 '송교1리',
 '큰말입구',
 '장외리.지긋말',
 '장외리',
 '숲풀안앞',
 '광평삼거리',
 '느티나무앞',
 '광평주유소',
 '광평리',
 '서신중학교후문',
 '매화리',
 '서신터미널',
 '상안리삼거리',
 '상안2리.SK주유소',
 '상안1.2리',
 '상안1리',
 '신흥사',
 '상안리.한국발효',
 '육일2리.코스코밸리',
 '육일리교차로',
 '육일리마을회관',
 '육일리.육교동입구',
 '사강복지회관',
 '사강시장',
 '사강정형외과',
 '봉가1리',
 '돼지고개.삼존리물미',
 '삼존리',
 '슬항3리',
 '구리섬휴게소',
 '외국인보호소',
 '마도사거리',
 '재래시장.마도삼거리',
 '두곡리',
 '쌍송리입구.은장고개',
 '남양육리.남양1동',
 '우림아파트',
 '경기남부수협.보건소',
 '남양성지',
 '남양사거리',
 '남양읍행정복지센터',
 '동수원남양병원',
 '북양1통.봉림사',
 '북양1통.염티고개',
 '양노3리',
 '양노리',
 '천천리입구',
 '삼미.쌍용아파트',
 '북양1통',
 '은장고개',
 '구리섬앞',
 '삼존리자동차성능시험장',
 '송산농협.사강정형외과',
 '서신',
 '목성머리',
 '살고지입구',
 '제부해수탕',
 '상안삼거리',
 '남양도서관.동광뷰엘',
 '남양동광뷰엘',
 '남양뉴타운4단지',
 '삼화리',
 '천산아파트',
 '비봉초등학교후문',
 '비봉중고등학교',
 '연화가든앞',
 '구포리',
 '야목리',
 '매송휴먼시아후문',
 '송라리',
 '원리',
 '송라1리',
 '왕재골입구',
 '비봉초등학교후문.삼화3리',
 '삼화리.천산아파트',
 '조암터미널',
 '조암터미널(경유)',
 '우정읍행정복지센터',
 '삼괴중고교',
 '조암한라비발디',
 '어은삼거리',
 '장안면행

In [95]:
len(st_nm)

1452

In [10]:
import pickle
with open("test.txt", "wb") as fp:   #Pickling
    pickle.dump(st_nm, fp)

In [19]:
RouteStation_latlon_sort = RouteStation_latlon.sort_values(['bus_line_no', 'bus_line_no_seq'], ascending=[True, True])

In [21]:
line_no_arr = RouteStation_latlon_sort['bus_line_no'].unique()

In [24]:
all_line_nm = []

for i in line_no_arr:
    tmp = RouteStation_latlon_sort[RouteStation_latlon_sort['bus_line_no']==i]
    all_line_nm.append(tmp['station_nm'].unique())

all_line_nm

[array(['조암터미널', '조암농협', '조암GS주유소', '어은삼거리', '어은4리', '장안면행정복지센터', '사랑리',
        '덕다2리앞', '덕다리입구', '독정1리', '독정삼거리', '장안9리입구', '장명초등학교', '장안6리',
        '장안3리', '장안8리', '장명초분교', '장안제일교회', '장안6리.물구리', '덕다2리앞(구래)',
        '삼괴중고교', '우정읍행정복지센터'], dtype=object),
 array(['조암터미널', '조암리(경유)', '어은삼거리', '어은4리', '장안면행정복지센터', '사랑리', '덕다2리앞',
        '덕다리입구', '독정1리', '독정삼거리', '장안9리입구', '장명초등학교', '장안6리', '장안3리',
        '장안8리', '장명초분교', '장안제일교회', '장안4리입구', '장안4리(고잔)종점', '장안6리.물구리',
        '덕다2리앞(구래)', '장안초등학교', '조암GS주유소'], dtype=object),
 array(['발안만세시장', '발안삼거리.바다마트', '발안리(경유)', '화성중고등학교', '화성시보건소앞', '장짐삼거리',
        '제약공단사원아파트앞', '지월리입구', '상신1리마을앞', '수목원', '상신6리', '구장리새터말입구', '상신리',
        '구장1리', '향남제약공단앞', '팔탄초등학교', '제약공단후문.구문천1리입구', '팔탄면행정복지센터', '구문천리',
        '마곡마을입구', '구문천공원', '구장삼거리', '발안산업단지', '구장사거리앞', '구문천1리진입로',
        '창곡주유소앞', '현동목장앞', '창곡3리.독골', '구문천삼거리', '우렁골입구', '구문천5리(회차)',
        '창곡2리교회앞', '창곡1리', '구문천리초록마을', '금산사입구', '구문천3리', '언창공단', '창곡1리입구',
        '청요사거리', '자안리', '우신공업앞

In [33]:
all_line_nm[0]

array(['조암터미널', '조암농협', '조암GS주유소', '어은삼거리', '어은4리', '장안면행정복지센터', '사랑리',
       '덕다2리앞', '덕다리입구', '독정1리', '독정삼거리', '장안9리입구', '장명초등학교', '장안6리',
       '장안3리', '장안8리', '장명초분교', '장안제일교회', '장안6리.물구리', '덕다2리앞(구래)',
       '삼괴중고교', '우정읍행정복지센터'], dtype=object)

In [26]:
tmp2 = list(all_line_nm[1])

In [32]:
tmp2[0].strip('\'')

'조암터미널'

In [94]:
len(all_line_nm)

219

In [108]:
nm_hash= {}

for i in st_nm:
    nm_hash[i] = -1

In [110]:
for i in range(len(all_line_nm)):
    for j in range(len(all_line_nm[i])):
        if all_line_nm[i][j] in list(nm_hash.keys()):
            nm_hash[all_line_nm[i][j]] += 1

In [112]:
import operator

In [115]:
sorted_nm_hash = sorted(nm_hash.items(), key=operator.itemgetter(1), reverse = True)

In [121]:
len(nm_hash.keys())

1452

In [126]:
sorted_nm_hash

[('사강정형외과', 48),
 ('조암터미널', 46),
 ('발안삼거리.바다마트', 43),
 ('사강종점', 40),
 ('사강복지회관', 37),
 ('발안만세시장', 37),
 ('사강시장', 36),
 ('화성중고등학교', 31),
 ('신창미션힐.송화초교', 31),
 ('KEB하나은행발안지점', 30),
 ('화성시보건소앞', 30),
 ('화성중앙병원.발안초등학교', 28),
 ('발안초등학교', 27),
 ('향남읍행정복지센터', 27),
 ('남양중고교입구', 26),
 ('남양중고입구.기아남양점', 26),
 ('기업은행', 26),
 ('남양뉴타운.양우내안애', 25),
 ('양우내안애', 25),
 ('병점역사거리', 25),
 ('화성소방서.향남홈플러스', 24),
 ('화성시청', 24),
 ('조암삼거리(경유)', 23),
 ('대광아파트', 23),
 ('공군부대입구', 23),
 ('한미약품.푸르지오', 23),
 ('예당고사거리', 23),
 ('홈플러스.벌말초교', 23),
 ('우남퍼스트빌', 22),
 ('송산중학교', 22),
 ('단독주택단지', 21),
 ('방축리', 20),
 ('지월리입구', 20),
 ('수목원', 20),
 ('구장리새터말입구', 20),
 ('상록.테크노밸리.GS자이', 20),
 ('바다마트', 20),
 ('남양사거리', 19),
 ('신영지웰', 19),
 ('장짐삼거리', 19),
 ('팔탄초등학교', 19),
 ('구장1리(새터말.율암리)', 19),
 ('구장1리', 19),
 ('봉가1리', 18),
 ('외국인보호소', 18),
 ('우림아파트', 18),
 ('발안사거리.평리', 18),
 ('팔탄면행정복지센터', 18),
 ('리베라CC', 18),
 ('전하리교회.우미제일', 18),
 ('돼지고개.삼존리물미', 17),
 ('구리섬휴게소', 17),
 ('경기남부수협.보건소', 17),
 ('남양성지', 17),
 ('구리섬앞', 17),
 ('삼존리자동차성능시험장'

In [105]:
for a in range(len(all_line_nm)):
    

dict_keys(['ee'])

In [132]:
all_line_nm = []

for i in line_no_arr:
    tmp = RouteStation_latlon_sort[RouteStation_latlon_sort['bus_line_no']==i]
    all_line_nm.append(i)
    all_line_nm.append(tmp['station_nm'].unique())

all_line_nm

['1', array(['조암터미널', '조암농협', '조암GS주유소', '어은삼거리', '어은4리', '장안면행정복지센터', '사랑리',
        '덕다2리앞', '덕다리입구', '독정1리', '독정삼거리', '장안9리입구', '장명초등학교', '장안6리',
        '장안3리', '장안8리', '장명초분교', '장안제일교회', '장안6리.물구리', '덕다2리앞(구래)',
        '삼괴중고교', '우정읍행정복지센터'], dtype=object), '1-1', array(['조암터미널', '조암리(경유)', '어은삼거리', '어은4리', '장안면행정복지센터', '사랑리', '덕다2리앞',
        '덕다리입구', '독정1리', '독정삼거리', '장안9리입구', '장명초등학교', '장안6리', '장안3리',
        '장안8리', '장명초분교', '장안제일교회', '장안4리입구', '장안4리(고잔)종점', '장안6리.물구리',
        '덕다2리앞(구래)', '장안초등학교', '조암GS주유소'], dtype=object), '10', array(['발안만세시장', '발안삼거리.바다마트', '발안리(경유)', '화성중고등학교', '화성시보건소앞', '장짐삼거리',
        '제약공단사원아파트앞', '지월리입구', '상신1리마을앞', '수목원', '상신6리', '구장리새터말입구', '상신리',
        '구장1리', '향남제약공단앞', '팔탄초등학교', '제약공단후문.구문천1리입구', '팔탄면행정복지센터', '구문천리',
        '마곡마을입구', '구문천공원', '구장삼거리', '발안산업단지', '구장사거리앞', '구문천1리진입로',
        '창곡주유소앞', '현동목장앞', '창곡3리.독골', '구문천삼거리', '우렁골입구', '구문천5리(회차)',
        '창곡2리교회앞', '창곡1리', '구문천리초록마을', '금산사입구', '구문천3리', '언창공단', '창곡1리입구',
        '청요사거리

In [140]:
arr2 = []
for i in st_nm:
    arr2.append(i)
    arr2.append(RouteStation_latlon[RouteStation_latlon['station_nm']==i]['bus_line_no'].unique())

In [166]:
arr2[1998:]

['나래울',
 array(['709', 'H6(A)', 'H6(B)', '17-1'], dtype=object),
 '모아미래도.자연앤',
 array(['709', 'H6(A)', 'H6(B)', '17-1'], dtype=object),
 '동양파라곤',
 array(['709', '19-1', '17-1'], dtype=object),
 '새강주공휴먼시아',
 array(['709-1', '19', '19-2', '19-3', 'H2', '27-2'], dtype=object),
 '나루마을상가',
 array(['709-1', 'H5'], dtype=object),
 '우림필유',
 array(['709-1', '19-2', '50-2(금강)', '50-4', '50-7', '50-4(대광1차)'],
       dtype=object),
 '화성미디어센터',
 array(['H4', 'H5', '17-1'], dtype=object),
 '동탄보건지소.우체국',
 array(['H4', 'H5', 'H6(A)', '27-1'], dtype=object),
 '대우푸르지오.예당고등학교',
 array(['H4', 'H5', 'H6(A)', 'H6(B)', '11-3', '27-1'], dtype=object),
 '솔빛동탄경남아너스빌',
 array(['H5', '17-1'], dtype=object),
 '동탄복합문화센터',
 array(['H5', '17-1'], dtype=object),
 '솔빛마을상가',
 array(['H5'], dtype=object),
 '쌍용예가',
 array(['H5', '19-2', '19-3', 'H2', '27-2'], dtype=object),
 '한마음초사거리',
 array(['H6(A)', 'H6(B)', '27-1'], dtype=object),
 '노작마을',
 array(['H6(A)'], dtype=object),
 '포스코정문앞',
 array(['H6(A)'], dtype=object),
 '

상위 5개 버스노선
```
720-3
116-3
116-2
721
712
```

In [38]:
# line1.sample()

,Unnamed: 0,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no,govern,lat,lng,이비노선ID,표준노선ID
3953,13847,97856,234000068,720-3,81,동탄이마트.한림대병원,233001618,37684.0,화성시,37.214033,127.077933,234000068.0,41002090.0


In [83]:
line1 = RouteStation_latlon_sort[RouteStation_latlon_sort['bus_line_no'] == '720-3']
tmp1 = []
for i in range(len(line1)):
    tmp1.append((line1.iloc[i]['lat'], line1.iloc[i]['lng']))

In [84]:
line2 = RouteStation_latlon_sort[RouteStation_latlon_sort['bus_line_no'] == '116-3']
tmp2 = []
for i in range(len(line2)):
    tmp2.append((line2.iloc[i]['lat'], line2.iloc[i]['lng']))

In [85]:
line3 = RouteStation_latlon_sort[RouteStation_latlon_sort['bus_line_no'] == '116-2']
tmp3 = []
for i in range(len(line3)):
    tmp3.append((line3.iloc[i]['lat'], line3.iloc[i]['lng']))

In [86]:
line4 = RouteStation_latlon_sort[RouteStation_latlon_sort['bus_line_no'] == '721']
tmp4 = []
for i in range(len(line4)):
    tmp4.append((line4.iloc[i]['lat'], line4.iloc[i]['lng']))

In [87]:
line5 = RouteStation_latlon_sort[RouteStation_latlon_sort['bus_line_no'] == '712']
tmp5 = []
for i in range(len(line5)):
    tmp5.append((line5.iloc[i]['lat'], line5.iloc[i]['lng']))

In [88]:
# All_top_busline[1][1] = (3, 3)
# All_top_busline[1][2]= (2,3)

In [89]:
All_top_busline = []
All_top_busline.append(tmp1)
All_top_busline.append(tmp2)
All_top_busline.append(tmp3)
All_top_busline.append(tmp4)
All_top_busline.append(tmp5)

# alpha = 0.005

# for i in range(4):
#     for j in range(len(All_top_busline[i+1]))
#         All_top_busline.append()

In [91]:
alpha = 0.005
Busline_top_ = folium.Map(location = [37.2353333,127.0621])
folium.PolyLine(All_top_busline[0], color="red", weight=2.5, opacity=1).add_to(Busline_top_)
# folium.PolyLine(All_top_busline[1], color="blue", weight=2.5, opacity=1).add_to(Busline_top_)
# folium.PolyLine(All_top_busline[2], color="yellow", weight=2.5, opacity=1).add_to(Busline_top_)
# folium.PolyLine(All_top_busline[3], color="violet", weight=2.5, opacity=1).add_to(Busline_top_)
# folium.PolyLine(All_top_busline[4], color="pink", weight=2.5, opacity=1).add_to(Busline_top_)
Busline_top_